In this project, I am gonna implement TextRank algorithm and calculate keyword score of each word. spaCy is used to get POS tag of words. 

In TextRank each word is considered as a node. w1,w2,...wn are n words. I set the window size as k. That is [w1, w2, …, w_k], [w2, w3, …, w_{k+1}], [w3, w4, …, w_{k+2}]. Any two-word pairs in a window are considered have an undirected edge. We take [time, Wandering, Earth, feels, throwback, eras, filmmaking] as the example, and set the window size k=4, so we get 4 windows, [time, Wandering, Earth, feels], [Wandering, Earth, feels, throwback], [Earth, feels, throwback, eras], [feels, throwback, eras, filmmaking].For window [time, Wandering, Earth, feels], any two words pair has an undirected edge. So we get (time, Wandering), (time, Earth), (time, feels), (Wandering, Earth), (Wandering, feels), (Earth, feels).

Since most of the words like preposition in a sentence have little semantic value. In this project I only consider only consider the words with NOUN, PROPN, VERB POS tags.

In [2]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import numpy as np
from collections import OrderedDict

In [3]:
nlp = spacy.load('en_core_web_sm')

In [56]:
class keywords():
    def __init__(self):
        self.d = 0.85 #damping coefficient, usually is .85
        self.diff = 1e-5 #convergence threshold
        self.steps = 20 #iteration steps
        self.node_weight = None # keywords and its weight
    '''define stop words'''
    def stopwords(self,stopwords):
        for token in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[token]
            lexeme.is_stop = True

    '''get words with some POS tags'''
    def get_words(self, doc, pos, lower):
        sent = []
        for sentence in doc.sents:
            word = []
            for token in sentence:
                if token.pos_ in pos and token.is_stop is False:
                    if lower is True:
                        word.append(token.text.lower())
                    else:
                        word.append(token.text)
            sent.append(word)
        return sent

    '''built word:apperance_order dictionary'''
    def get_vocab(self, doc):
        vocab = OrderedDict()
        i = 0
        for sent in doc:
            for word in sent:
                if word not in vocab:
                    vocab[word] = i
                    i += 1               
        return vocab

    '''build token pair with certain lenght window'''
    def get_pairs(self, window_len, doc):
        token_pair = []
        for sent in doc:
            for i,word in enumerate(sent):
                for j in range(i+1, i+window_len):
                    if j >= len(sent):
                        break
                    pair = (word, sent[j])
                    if pair not in token_pair:
                        token_pair.append(pair)
        return token_pair

    def symmetrize(self, m):
        return m+m.T-np.diag(m.diagonal())

    '''contruct token directed matrix'''
    def get_matrix(self, vocab, token_pairs):
        size = len(vocab)
        m = np.zeros((size,size), dtype = 'float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            m[i][j] = 1

        m = self.symmetrize(m)
        
        '''normalize matrix'''
        s = np.sum(m, axis = 0)
        normalized = np.divide(m, s, where=s!= 0 ) #ignore 0 element in norm

        return normalized

    '''main function to calculate weight for each word'''
    def analyze(self, doc, pos=['NOUN', 'PROPN', 'VERB'], window_len = 3, lower = False, stopwords = list()):

        doc = nlp(doc)
        self.stopwords(stopwords)
        sents = self.get_words(doc, pos, lower)
        vocab = self.get_vocab(sents)
        token_pairs = self.get_pairs(window_len, sents)
        m = self.get_matrix(vocab, token_pairs)

        w = np.array([1]*len(vocab))

        pre_value = 0
        for i in range(self.steps):
            w = (1-self.d) + self.d * np.dot(m, w)
            if abs(pre_value-sum(w)) < self.diff:
                break
            else:
                pre_value = sum(w)

        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = w[index]

        self.node_weight = node_weight


    '''print top 10 keywords'''
    def get_keywords(self, num = 10):
        node_weight = OrderedDict(sorted(self.node_weight.items(), key = lambda t:t[1], reverse = True))
        for i, (key,value) in enumerate(node_weight.items()):
            print(key+':'+str(value))
            if i > num:
                break

In [57]:
text = '''
The Wandering Earth, described as China’s first big-budget science fiction thriller, quietly made it onto screens at AMC theaters in North America this weekend, and it shows a new side of Chinese filmmaking — one focused toward futuristic spectacles rather than China’s traditionally grand, massive historical epics. At the same time, The Wandering Earth feels like a throwback to a few familiar eras of American filmmaking. While the film’s cast, setting, and tone are all Chinese, longtime science fiction fans are going to see a lot on the screen that reminds them of other movies, for better or worse.
'''

test1 = keywords()


In [58]:
test1.analyze(text)

get top 10 keywords

In [59]:
test1.get_keywords()

China:1.500404974489796
science:1.4267889030612242
Earth:1.3999011479591834
fiction:1.3888424744897958
filmmaking:1.3209424603174602
screen:1.1285119047619045
setting:1.0898065476190473
lot:1.0729836309523808
tone:1.0565582482993197
feels:1.0121850198412696
going:1.0118356717687074
focused:1.0026442035147392


referece:Xu Liang https://github.com/BrambleXu/news-graph?source=post_page-----c0bae21bcec0----------------------